In [1]:
%load_ext autoreload
%autoreload 2

In [48]:
import sys, os,git
from pathlib import Path
from pprint import pprint
import subprocess


os.environ['PATH'] = '/g/data/vk83/apps/payu/1.1.3/bin:' + os.environ['PATH']
print("PATH:", os.environ['PATH'])
!python -c 'import payu; print(payu)'

try:
    import yaml
    import f90nml  # from https://f90nml.readthedocs.io/en/latest/
except ImportError:  # BUG: don't get this exception if payu module loaded, even if on python 2.6.6
    print('\nFatal error: modules not available.')
    print('On NCI, do the following and try again:')
    print('   module use /g/data/hh5/public/modules; module load conda/analysis3\n')
    raise

PATH: /g/data/vk83/apps/payu/1.1.3/bin:/g/data/vk83/apps/payu/1.1.3/bin:/g/data/vk83/apps/payu/1.1.3/bin:/g/data/hh5/public/apps/miniconda3/envs/analysis3-unstable/bin:/g/data/hh5/public/apps/miniconda3/condabin:/opt/pbs/default/bin:/opt/nci/bin:/opt/bin:/opt/Modules/v4.3.0/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/local/pbs/bin
<module 'payu' from '/g/data/vk83/apps/payu/1.1.3/lib/python3.9/site-packages/payu/__init__.py'>


In [3]:
# ======================================================
# from https://gist.github.com/paulkernstock/6df1c7ad37fd71b1da3cb05e70b9f522
from yaml.representer import SafeRepresenter


class LiteralString(str):
    pass


def change_style(style, representer):
    def new_representer(dumper, data):
        scalar = representer(dumper, data)
        scalar.style = style
        return scalar
    return new_representer

represent_literal_str = change_style('|', SafeRepresenter.represent_str)
yaml.add_representer(LiteralString, represent_literal_str)
# ======================================================

In [7]:
dir_ensemble = '/g/data/tm70/ml0072/COMMON/ensemble'
sys.path.append(dir_ensemble)

dir_utils = '/g/data/tm70/ml0072/COMMON/om3-utils/om3utils'
sys.path.append(dir_utils)

In [8]:
yaml_text = 'ensemble.yaml'
yamlfile = Path(dir_ensemble,yaml_text)
indata = yaml.load(open(yamlfile, 'r'), Loader=yaml.SafeLoader)
indata

{'template': 'test/1deg_jra55_ryf',
 'startfrom': 'rest',
 'nruns': 0,
 'namelists': {'ice/cice_in.nml': {'shortwave_nml': {'albicei': [0.3,
     0.44,
     0.5]}}}}

In [9]:
template = indata['template']
print(f'template: {template}')
templatepath = Path(dir_ensemble,template)
print(f'templatepath: {templatepath}')
templaterepo = git.Repo(templatepath)
print(templaterepo)
print(f'current branch: {templaterepo.active_branch.name}')

template: test/1deg_jra55_ryf
templatepath: /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf
<git.repo.base.Repo '/g/data/tm70/ml0072/COMMON/ensemble/.git/modules/test/1deg_jra55_ryf'>
current branch: master


In [10]:
startfrom = str(indata['startfrom']).strip().lower().zfill(3)
startfrom

'rest'

In [50]:
from ruamel.yaml import YAML
from ruamel.yaml.comments import CommentedMap

yaml = YAML()
yaml.preserve_quotes = True

def read_config_yaml(exp_path)
    configpath = Path(exp_path,'config.yaml')
    with open(configpath,'r') as f:
        config = yaml.load(f)
    return config

def write_config_yaml(config)
if isinstance(config, CommentedMap):
    # Update specific parameters
    config['jobname'] = exp_name
    with open(configpath,'w') as f:
        yaml.dump(config,f)
                        

SyntaxError: expected ':' (2747741210.py, line 4)

In [49]:
ensemble = []
branch = 'master'
branch_new = 'ctrl'




for fname, nmls in indata['namelists'].items():
    print(f'namelists:{fname,nmls}')
    for group, names in nmls.items():
        print(f"nmls: {group,names}")
        for name,values in names.items():
            print(f"names: {name,values}")
            for v in values:
                print(f'value: {v}')
                exp_path = os.path.join(dir_ensemble,'_'.join([template,name,str(v)]))
                print(exp_path)
                rel_path = '_'.join([template,name,str(v)])
                print(rel_path)
                exp_name = os.path.basename(rel_path)
                print(exp_name)

                # if os.path.exists(exp_path):
                #     ensemble.append(exp_path)
                #     continue

                # check if paramter change differs from the template
                
                parser = f90nml.Parser()
                nml = parser.read(Path(templatepath,fname))
                #print(nml)
                print(nml[group][name])
                if v == nml[group][name]:
                    print(' -- not creating ', rel_path, ' - parameters are identical to the template ',template,'\n')
                    continue
                else:
                    print(' -- creating new exp ', rel_path,'\n')

                if os.path.exists(exp_path):
                    print(' -- not creating ', rel_path, ' - already exists!','\n')
                    
                    
                else:
                    print(f'clone template - payu clone!','\n')
                    # clone template payu clone
                    # payu clone -B master -b ctrl test/1deg_jra55_ryf test/1deg_jra55_ryf_test
                    command = f'payu clone -B {branch} -b {branch_new} {templatepath} {exp_path}'
                    test = subprocess.run(command, shell=True, check=False, capture_output=True, text=True)
                    
                nml_tmp = parser.read(Path(exp_path,fname))
                nml_tmp[group][name] = v
                print(f'changed value: {nml_tmp[group][name]}')
                with open(Path(exp_path,fname),'w') as nml_file:
                    f90nml.write(nml_tmp,nml_file)
                    
                
#ensemble

namelists:('ice/cice_in.nml', {'shortwave_nml': {'albicei': [0.3, 0.44, 0.5]}})
nmls: ('shortwave_nml', {'albicei': [0.3, 0.44, 0.5]})
names: ('albicei', [0.3, 0.44, 0.5])
value: 0.3
/g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.3
test/1deg_jra55_ryf_albicei_0.3
1deg_jra55_ryf_albicei_0.3
0.44
 -- creating new exp  test/1deg_jra55_ryf_albicei_0.3 

clone template - payu clone! 

changed value: 0.3
{'queue': 'normal', 'walltime': '3:00:00', 'jobname': '1deg_jra55_ryf', 'mem': '1000GB', 'name': 'common', 'model': 'access-om2', 'input': '/g/data/ik11/inputs/access-om2/input_20201102/common_1deg_jra55', 'submodels': [{'name': 'atmosphere', 'model': 'yatm', 'exe': '/g/data/ik11/inputs/access-om2/bin/yatm_d750b4b.exe', 'input': ['/g/data/ik11/inputs/access-om2/input_20201102/yatm_1deg', '/g/data/ik11/inputs/JRA-55/RYF/v1-4'], 'ncpus': 1}, {'name': 'ocean', 'model': 'mom', 'exe': '/g/data/ik11/inputs/access-om2/bin/fms_ACCESS-OM_730f0bf_libaccessom2_d750b4b.x', 'input': '/

In [46]:
config

{'collate': {'exe': '/g/data/ik11/inputs/access-om2/bin/mppnccombine', 'mem': '30GB', 'ncpus': 4, 'queue': 'normal', 'restart': True, 'walltime': 3600}, 'env': {'UCX_LOG_LEVEL': 'error'}, 'input': '/g/data/ik11/inputs/access-om2/input_20201102/common_1deg_jra55', 'jobname': '1deg_jra55_ryf_albicei_0.5', 'mem': '1000GB', 'model': 'access-om2', 'mpirun': '--mca io ompio --mca io_ompio_num_aggregators 1', 'name': 'common', 'platform': {'nodesize': 48}, 'qsub_flags': '-W umask=027', 'queue': 'normal', 'restart_freq': 1, 'runlog': True, 'stacksize': 'unlimited', 'submodels': [{'exe': '/g/data/ik11/inputs/access-om2/bin/yatm_d750b4b.exe', 'input': ['/g/data/ik11/inputs/access-om2/input_20201102/yatm_1deg', '/g/data/ik11/inputs/JRA-55/RYF/v1-4'], 'model': 'yatm', 'name': 'atmosphere', 'ncpus': 1}, {'exe': '/g/data/ik11/inputs/access-om2/bin/fms_ACCESS-OM_730f0bf_libaccessom2_d750b4b.x', 'input': '/g/data/ik11/inputs/access-om2/input_20201102/mom_1deg', 'model': 'mom', 'name': 'ocean', 'ncpus'

In [17]:
test

CompletedProcess(args='payu clone -B master -b ctrl /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5', returncode=0, stdout='Cloned repository from /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf to directory: /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5\nCreated and checked out new branch: ctrl\nlaboratory path:  /scratch/tm70/ml0072/access-om2\nbinary path:  /scratch/tm70/ml0072/access-om2/bin\ninput path:  /scratch/tm70/ml0072/access-om2/input\nwork path:  /scratch/tm70/ml0072/access-om2/work\narchive path:  /scratch/tm70/ml0072/access-om2/archive\nUpdated metadata. Experiment UUID: 7787c97c-833f-4ae4-bb3b-f03d2e71327c\nAdded archive symlink to /scratch/tm70/ml0072/access-om2/archive/1deg_jra55_ryf_albicei_0.5-ctrl-7787c97c\nTo change directory to control directory run:\n  cd /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5\n', stderr='')

In [18]:
test.returncode

0

In [62]:
os.getcwd()

'/home/563/ml0072'

In [79]:
turningangle = [fname, group, name] == ['ice/cice_in.nml', 'dynamics_nml', 'turning_angle']
turningangle
name

'albicei'

In [ ]:
payu clone -B master -b ctrl 1deg_jra55_ryf_test

In [106]:
!payu --version

payu 1.0.19


In [110]:
test

CompletedProcess(args='!payu clone -B master -b ctrl /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5', returncode=127, stdout='', stderr='/bin/sh: !payu: command not found\n')

In [112]:
test

CompletedProcess(args='payu clone -B master -b ctrl /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5', returncode=2, stdout='', stderr="usage: payu [-h] [--version]\n            {archive,build,collate,ghsetup,init,list,profile,push,run,setup,sweep,sync}\n            ...\npayu: error: argument {archive,build,collate,ghsetup,init,list,profile,push,run,setup,sweep,sync}: invalid choice: 'clone' (choose from 'archive', 'build', 'collate', 'ghsetup', 'init', 'list', 'profile', 'push', 'run', 'setup', 'sweep', 'sync')\n")

In [3]:
!payu clone

usage: payu [-h] [--version]
            {archive,build,collate,ghsetup,init,list,profile,push,run,setup,sweep,sync}
            ...
payu: error: argument {archive,build,collate,ghsetup,init,list,profile,push,run,setup,sweep,sync}: invalid choice: 'clone' (choose from 'archive', 'build', 'collate', 'ghsetup', 'init', 'list', 'profile', 'push', 'run', 'setup', 'sweep', 'sync')


In [6]:
!python --version

Python 3.10.14


In [5]:
!python -c 'import payu; print(payu)'

<module 'payu' from '/g/data/hh5/public/apps/miniconda3/envs/analysis3-unstable/lib/python3.10/site-packages/payu/__init__.py'>


In [1]:
import sys
print(sys.executable)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/bin/python


In [2]:
%env PATH=/g/data/vk83/modules/bin:$PATH

env: PATH=/g/data/vk83/modules/bin:$PATH


PATH: /g/data/vk83/apps/payu/1.1.3/bin:/g/data/vk83/modules/bin:/g/data/vk83/modules/bin:$PATH


In [8]:
!payu clone

usage: payu clone [-h] [--model MODEL_TYPE] [--config CONFIG_PATH]
                  [--laboratory LAB_PATH] [-k] [--branch BRANCH]
                  [--new-branch NEW_BRANCH_NAME] [--restart RESTART_PATH]
                  [--parent-experiment PARENT_EXPERIMENT]
                  repository local_directory
payu clone: error: the following arguments are required: repository, local_directory
